In [ ]:
#

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "/home/cogknit/project/hatespeech/input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 20000
maxlen = 100



sample_submission.csv
test.csv
test_labels.csv
train.csv



Using TensorFlow backend.


In [2]:

train = pd.read_csv("/home/cogknit/project/hatespeech/input/train.csv")
test = pd.read_csv("/home/cogknit/project/hatespeech/input/test.csv")
train = train.sample(frac=1)

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values




In [3]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)



In [4]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model




In [6]:
model = get_model()
batch_size = 1
epochs = 1
file_path="weights_base.best.hdf5"


In [ ]:
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

In [7]:
model.load_weights(file_path)

y_test = model.predict(X_te[:1])

In [8]:
y_test

array([[0.9947843 , 0.14364685, 0.9381442 , 0.03921968, 0.83358014,
        0.17879346]], dtype=float32)

In [ ]:

sample_submission = pd.read_csv("/home/cogknit/project/hatespeech/input/sample_submission.csv")
# sample_submission[list_classes] = y_test
# sample_submission.to_csv("baseline.csv", index=False)

# Inference

In [9]:
list_sentences_train=np.array(["It could be, if he was given a leaked text of the speech"])
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]


In [10]:


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_data = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)


In [11]:
model.load_weights(file_path)

y_test = model.predict(X_data)

In [12]:
y_test

array([[0.07231109, 0.00014522, 0.0053334 , 0.00117047, 0.00820538,
        0.00210451]], dtype=float32)

In [13]:
pridiction=y_test[0]
result = dict(zip(list_classes, pridiction))
result

{'identity_hate': 0.00210451,
 'insult': 0.008205385,
 'obscene': 0.0053333994,
 'severe_toxic': 0.00014521879,
 'threat': 0.0011704676,
 'toxic': 0.07231109}